In [1]:
import pandas as pd
import os
import datetime

In [2]:
def open_file(temp_path):
    try:
        if '.csv' in temp_path:
            return pd.read_csv(temp_path, encoding='utf-8')
        if '.xlsx' in temp_path:
            return pd.read_excel(temp_path)
    except ValueError:
        raise ValueError(temp_path)

## 날짜 유혀성 확인
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        return False

In [3]:
## 온도 데이터 위치
root_path = 'additional/dust'
save_path = 'dust_total.csv'
assert os.path.isdir(root_path), "온도 데이터가 있는 폴더 설정 하세요."

In [4]:
## 데이터 확인
filelist = [item for item in os.listdir(root_path) if '.csv' in item or '.xlsx' in item]
print(filelist)

['일별평균대기오염도_2017.xlsx', '일별평균대기오염도_2015.xlsx', '일별평균대기오염도_2019.xlsx', '일별평균대기오염도_2016.xlsx', '일별평균대기오염도_2018.xlsx']


In [5]:
merge_dict = {
    'date':[],
    'val':[],
}

for filename in filelist:
    temp_year = filename.split('.')[0].split('_')[1]
    
    ## 데이터 하나씩 불러오기
    df = open_file(os.path.join(root_path, filename))
    
    ##데이터 추출
    temp_df = df[['측정일시', '미세먼지(㎍/㎥)']].groupby('측정일시').mean()
    temp_df_date = temp_df.index
    temp_df_val = temp_df.values
    
    for temp_date, temp_data in zip(temp_df_date, temp_df_val):
        temp_data = temp_data[0]
        temp_date = str(temp_date)
        temp_date = "{}-{}-{}".format(temp_date[:4], temp_date[4:6], temp_date[6:])

        ## 날짜 형식 확인
        if validate(temp_date):
            merge_dict['date'].append(temp_date)
            merge_dict['val'].append(temp_data)
    

In [6]:
merge_df = pd.DataFrame(merge_dict)

In [7]:
merge_df = merge_df.sort_values('date')

In [8]:
merge_df = merge_df.reset_index(drop=True)

In [9]:
merge_df.columns = ['date', 'dust']

In [10]:
## 저장하기
merge_df.to_csv(save_path, index=None)